In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
from autodp.calibrator_zoo import eps_delta_calibrator,generalized_eps_delta_calibrator, ana_gaussian_calibrator
from autodp import rdp_bank
from autodp.mechanism_zoo import ExactGaussianMechanism, PureDP_Mechanism,SubsampleGaussianMechanism, GaussianMechanism, ComposedGaussianMechanism, LaplaceMechanism
from autodp.transformer_zoo import Composition, AmplificationBySampling, ComposeGaussian

In [2]:
calibrate = eps_delta_calibrator()
ana_calibrate = ana_gaussian_calibrator()
eps = 0.1
delta = 1e-6

mech1 = calibrate(ExactGaussianMechanism,eps,delta,[0,100],name='GM')
mech2 = ana_calibrate(ExactGaussianMechanism, eps, delta, name='Ana_GM')
print(mech1.name, mech1.params, mech1.get_approxDP(delta))
print(mech2.name, mech2.params, mech2.get_approxDP(delta))

GM {'sigma': 36.30468875626966} 0.10000000492561087
Ana_GM {'sigma': 36.304691899114694} 0.09999999565548537


In [4]:
coeff = 20
general_calibrate = generalized_eps_delta_calibrator()
params = {}
params['sigma'] = None
params['coeff'] = 20

mech3 = general_calibrate(ComposedGaussianMechanism, eps, delta, [0,1000],params=params,para_name='sigma', name='Composed_Gaussian')
print(mech3.name, mech3.params, mech3.get_approxDP(delta))
#coeff is the number of composition. The calibrator calibrates the noise with other parameters (coeff) fixed.
#[0,1000] is the range of sigma.

Composed_Gaussian {'sigma': 176.4880194030161, 'coeff': 20} 0.0999999989438041


In [5]:
calibrate = generalized_eps_delta_calibrator()

eps = 1
delta = 0
mech = calibrate(LaplaceMechanism,eps,delta,[0,100],name='Laplace')
print(mech.name, mech.params, mech.get_approxDP(delta))
#[0, 100] is the range of laplace parameter b.
# to calibrate the noise for the composed Laplace mechanism, we can define a new composed Lapalce mechanism in 
# mechanism.zoo (similar with Example 2)

Laplace {'b': 1.0000000325263934} 0.9999999674736078


In [49]:
# The first step is to package this in a function where the input are the parameters,
# and the output is the mechanism object

def create_complex_mech(b):
    lm1 = LaplaceMechanism(b, name='LM1')
    lm2 = LaplaceMechanism(b * 9, name='LM2')

    # compose them with the transformation: compose.
    compose = Composition()
    composed_mech = compose([lm1, lm2], [1,1])
    return composed_mech

# next we can create it as a mechanism class, which requires us to inherit the base mechanism class,
#  which we import now

from autodp.autodp_core import Mechanism

class Complex_Mechanism(Mechanism):
    def __init__(self, b):
        super().__init__()
        self.b = b
        self.lm1 = LaplaceMechanism(b, name='LM1')
        self.lm2 = LaplaceMechanism(b * 9, name='LM2')

        # compose them with the transformation: compose.
        compose = Composition()
        composed_mech = compose([self.lm1, self.lm2], [1,1])
        # The following will set the function representation of the complex mechanism
        # to be the same as that of the mech
        self.set_all_representation(composed_mech)


# Now one can calibrate the mechanism to achieve a pre-defined privacy budget

# Let's say we are to calibrate the noise to the following privacy budget
eps_budget = 1
delta_budget= 1e-3

# declare a general_calibrate "Calibrator"

general_calibrate = eps_delta_calibrator()

# Set those parameters that we want to fix and leave 
# params = {}
# params['b'] = None

# Next, jussts call `general_calibrate` to find the right value of `eps` 
# to give us the mechanism that achieves the required privacy-budget
mech = general_calibrate(Complex_Mechanism, eps_budget, delta_budget, [0,50])

print(mech.b, mech.get_approxDP(delta_budget))

1.106693887226024 0.9999993005946465


/remote/idiap.svm/temp.social02/sajadmanesh/conda/envs/main/lib/python3.9/site-packages/scipy/optimize/optimize.py:2621: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom


In [70]:
lm = LaplaceMechanism(b=0)
lm.params['b'] = 1.0
lm.get_approxDP(0)

ZeroDivisionError: division by zero

In [74]:
lm = LaplaceMechanism(b=0)
lm.propagate_updates((1,0), type_of_update='approxDP')

In [75]:
lm.params

{'b': 0}

In [76]:
lm.get_approxDP(0)

ZeroDivisionError: division by zero